In [1]:
import json
import os
import time
import random
from openai import OpenAI
import sys

In [134]:
df.head()

,mi_quality,transcript_id,video_title,video_url,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type
0,high,0,"NEW VIDEO: Brief intervention: ""Barbara""",https://www.youtube.com/watch?v=PaSKcfTmFEk,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN
1,high,0,"NEW VIDEO: Brief intervention: ""Barbara""",https://www.youtube.com/watch?v=PaSKcfTmFEk,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral
2,high,0,"NEW VIDEO: Brief intervention: ""Barbara""",https://www.youtube.com/watch?v=PaSKcfTmFEk,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN
3,high,0,"NEW VIDEO: Brief intervention: ""Barbara""",https://www.youtube.com/watch?v=PaSKcfTmFEk,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral
4,high,0,"NEW VIDEO: Brief intervention: ""Barbara""",https://www.youtube.com/watch?v=PaSKcfTmFEk,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN


In [139]:
import pandas as pd

df = pd.read_csv('AnnoMI-full.csv')
topic = ''
scripts = []
script = ''
for i in range(len(df)):
    
    row = df.iloc[i]
    if topic != row['topic']:
        if topic != '':
            scripts.append(script)
        topic = row['topic']
        script = ''
    script += f"{row['interlocutor']}:{row['utterance_text']}\n"
    

In [142]:
#api 설정
try:
    api_key = "your-key"
    if not api_key:
        raise ValueError("OPENAI_API_KEY 환경 변수가 설정되지 않았습니다.")
    client = OpenAI(api_key=api_key)
except ValueError as e:
    print(f"오류: {e}")
    sys.exit(1) # 오류 발생 시 스크립트 종료
except Exception as e:
    print(f"OpenAI 클라이언트 초기화 중 오류 발생: {e}")
    sys.exit(1)

In [143]:
#대화 생성 모델
DIALOGUE_MODEL_NAME = "gpt-5-mini"

In [144]:
#요약 및 상태 판단 모델
LOW_COST_MODEL_NAME = "gpt-5-mini"

In [145]:
#입력 JSON 파일
INPUT_JSON_FILE = "mi_evaluation_results.json"

In [146]:
#생성 스크립트 수
NUM_SCRIPTS_TO_GENERATE = 20

In [172]:
#대화 턴 수 (상담사10+내담자10 = 20턴)
MAX_DIALOGUE_TURNS = 10

In [173]:
#API 지연
DELAY_BETWEEN_CALLS = 0.1

In [174]:
#출력 파일 이름
OUTPUT_FILE = f"mi_multiturn_dataset_cactus_style_{DIALOGUE_MODEL_NAME}.jsonl"

In [175]:
#API 호출 함수
def call_openai_api(model_name, messages, max_tokens=150, temperature=0.7, attempt=1, max_attempts=3):
    """지정된 모델과 메시지로 OpenAI API를 호출합니다."""
    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=messages
        )
        if response.choices and response.choices[0].message:
            return response.choices[0].message.content.strip()
        else:
            print("API 응답 구조가 예상과 다릅니다.")
            return None
    except Exception as e:
        print(f"API 호출 오류 발생 ({model_name}, 시도 {attempt}/{max_attempts}, 오류 유형: {type(e).__name__}): {e}")
        if hasattr(e, 'response') and hasattr(e.response, 'text'):
             print(f"   오류 응답: {e.response.text}")
        if attempt < max_attempts:
            wait_time = DELAY_BETWEEN_CALLS * (2 ** attempt)
            print(f"{wait_time}초 후 재시도합니다... ({attempt+1}/{max_attempts})")
            time.sleep(wait_time)
            return call_openai_api(model_name, messages, max_tokens, temperature, attempt + 1, max_attempts)
        else:
            print(f"최대 재시도 횟수({max_attempts})를 초과했습니다.")
            return None

In [176]:
#사연 요약 함수
SUMMARIZATION_SYSTEM_PROMPT = """
당신은 동기부여 상담 전문가입니다. 내담자가 상담 신청 시 작성한 긴 사연을 읽고, 상담사가 상담 시작 전에 빠르게 파악할 수 있도록 핵심 문제와 상황을 1문장으로 요약해주세요.
- 내담자의 주요 호소 문제가 무엇인지 명확히 드러내야 합니다.
- 요약문은 객관적인 사실 위주로 작성합니다.
"""

BELIEF_PROMPT = """

## 작업: 동기 면담(MI) 클라이언트 프로필 요소 정의

제공된 내담자 사연([Client Narrative])을 면밀히 분석하고, 행동 변화를 유도하는 동기 면담(MI)의 핵심 구성 요소인 'goal'과 'belief'를 정의하십시오. 이 요소들은 상담사가 내담자의 상태 전이(State Transition)를 유도하고 일관성 있는 반응을 생성하는 데 사용됩니다 

## 정의 기준 및 형식

1.  **goal (변화 목표):**
    *   **정의:** 내담자가 행동 변화를 고려하도록 이끌 수 있는 **특정하고 핵심적인 이유**를 한 문장으로 작성해야 합니다 [5]. 이는 클라이언트의 더 큰 목표(예: 건강, 가족, 직업, 가치관)와 연결되어야 합니다 [6].
    *   **목표:** 상담사가 이 동기를 언급했을 때, 내담자가 '숙고 전 단계(Precontemplation)'에서 '숙고 단계(Contemplation)'로 진입하는 계기가 될 수 있도록 구체적이어야 합니다 [6].

2.  **belief (변화를 주저하게 하는 신념):**
    *   **정의:** 내담자가 자신의 행동 문제(Behavioral Problem)를 인정하더라도 변화를 **방해하거나 주저하게 만드는** 내면적인 신념이나 잘못된 생각을 한 문장으로 요약하여 작성해야 합니다 [5].
    *   **목표:** 이 신념은 내담자가 변화에 대해 '주저(Hesitate)'하거나 '의심(Doubt)'하는 행동을 유발하며, '숙고 단계(Contemplation)'에서 '준비 단계(Preparation)'로 나아가는 것을 가로막는 장애물 역할을 해야 합니다 [6, 7].

## 단계의 정의
Precontemplation:내담자는 행동 문제를 부정하거나 경시하며, 변화의 필요성을 인지하지 못하거나 과소평가합니다 이 시기는 정보 제공(Inform)이 가장 두드러지게 나타나며, 내담자는 이 단계에서 프로필 정보를 공유할 가능성이 높습니다,
Contemplation:내담자는 자신의 행동 문제를 인정하지만, 종종 내담자의 프로필에 포함된 신념(beliefs) 때문에 변화를 주저합니다,
Preparation:내담자가 행동 변화를 위한 계획을 세우기 시작하는 단계입니다. 내담자는 행동할 준비가 되었으며, 변화를 위한 구체적인 단계를 계획합니다,                    



## 출력 형식

분석된 내용을 반드시 다음 JSON 형식으로만 출력해야 합니다.


{
  "goal": "클라이언트가 변화하고자 하는 핵심 이유를 담은 한 문장.",
  "belief": "변화를 주저하게 만드는 핵심 신념을 요약한 한 문장."
}

belief가 드러나 있지 않는다면, 개연성있게 창작하세요


"""

def summarize_story(story_text):
    """저비용 모델을 사용해 긴 사연을 요약합니다."""
    print("사연 요약 중...")
    messages = [
        {"role": "system", "content": SUMMARIZATION_SYSTEM_PROMPT},
        {"role": "user", "content": story_text}
    ]
    summary = call_openai_api(LOW_COST_MODEL_NAME, messages, max_tokens=200, temperature=0.3)
    if summary:
        print(f"요약 완료: {summary}")
        return summary
    else:
        print("요약 실패. 원본 사연의 앞 100자를 대신 사용합니다.")
        return story_text[:100] + "..." # 요약 실패 시 대체 텍스트
    
def extract_belief(story_text):
    """저비용 모델을 사용해 긴 사연을 요약합니다."""
    print("사연 요약 중...")
    messages = [
        {"role": "system", "content": BELIEF_PROMPT},
        {"role": "user", "content": story_text}
    ]
    summary = call_openai_api(LOW_COST_MODEL_NAME, messages, max_tokens=200, temperature=0.3)
    return summary

In [177]:
#상태 전이 판단 함수
PRE_TO_CON_PROMPT = """
당신은 MI(동기 강화 상담) 전문가입니다. 주어진 상담 대화 기록, 특히 가장 최근의 상담사 발화를 분석하여, 내담자가 다음 단계로 전이하는 조건에 있는지 알려주세요

현재 내담자의 상태는 'Precontemplation' 입니다. 이는 
'내담자는 행동 문제를 부정하거나 경시하며, 변화의 필요성을 인지하지 못하거나 과소평가합니다' 의 상태입니다

상태 전이 조건 : 상담사가 내담자의 동기와 관련된 특정 이유를 언급하고 이를 통해 내담자가 동기 부여를 받을 때 Contemplation 진입합니다.

대화를 보았을 때 , 내담자의 상태가 Contemplation으로 전이한 상태인가요?

YES or NO 로 대답하세요

"""

CON_TO_PREP_PROMPT = """
당신은 MI(동기 강화 상담) 전문가입니다. 주어진 상담 대화 기록, 특히 가장 최근의 상담사 발화를 분석하여, 내담자가 다음 단계로 전이하는 조건에 있는지 알려주세요

현재 내담자의 상태는 'Contemplation' 입니다. 이는 
'내담자는 자신의 행동 문제를 인정하지만, 종종 내담자의 프로필에 포함된 신념(beliefs) 때문에 변화를 주저합니다' 의 상태입니다

상태 전이 조건 : 내담자의 프로필에 있는 주저하는 이유(신념)가 상담사에 의해 적절하게 다루어졌을 때 Preparation 진입합니다.

대화를 보았을 때 , 내담자의 상태가 Preparation으로 전이한 상태인가요?

YES or NO 로 대답하세요

"""

PREP_TO_TER_PROMPT = """
당신은 MI(동기 강화 상담) 전문가입니다. 주어진 상담 대화 기록, 특히 가장 최근의 상담사 발화를 분석하여, 내담자가 다음 단계로 전이하는 조건에 있는지 알려주세요

현재 내담자의 상태는 'Preparation' 입니다. 이는 
'내담자가 행동 변화를 위한 계획을 세우기 시작하는 단계입니다' 의 상태입니다

상태 전이 조건 : 내담자가 선호하는 변화 계획(preferred change plan)에 대해 논의했을 때 Termination 진입합니다.

대화를 보았을 때 , 내담자의 상태가 Termination으로 전이한 상태인가요?

YES or NO 로 대답하세요

"""

def judge_client_state_rule_based(current_state, conversation_history):
    """규칙 기반으로 내담자의 다음 상태를 판단합니다."""
    print(f"내담자 상태 전이 판단 중... (현재: {current_state})")
    system_prompt = ""
    next_state_if_yes = current_state
    if current_state == "Precontemplation":
        system_prompt = PRE_TO_CON_PROMPT
        next_state_if_yes = "Contemplation"
    elif current_state == "Contemplation":
        system_prompt = CON_TO_PREP_PROMPT
        next_state_if_yes = "Preparation"
    elif current_state == "Preparation":
        system_prompt = PREP_TO_TER_PROMPT
        next_state_if_yes = "Termination"
    else:
        return current_state # Preparation 단계 이상은 유지

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "다음 대화 기록을 바탕으로 상태 전이 조건을 판단해주세요:\n\n" + json.dumps(conversation_history, ensure_ascii=False, indent=2)}
    ]
    judgment = call_openai_api(LOW_COST_MODEL_NAME, messages, max_tokens=5, temperature=0.1)

    if judgment == "YES":
        print(f"상태 전이 조건 충족: {current_state} -> {next_state_if_yes}")
        return next_state_if_yes
    else:
        print(f"상태 전이 조건 불충족: {current_state} 유지.")
        return current_state
    
ACTION_SELECT_PROMPT = """
당신은 MI(동기 강화 상담) 전문가입니다. 내담자의 현재 상태는 {state}이며 그 상태는 {state_explain}을 의미합니다
주어진 대화 맥락을 고려하고 내담자의 현재 상태를 고려하여 내담자 고민 상황을 고려하여 actions 중에서 다음 행동을 선택하세요
내담자의 고민 상황 : {summary}
내담자의 대화의 목표는 summary를 해결하기 위한 상담을 받는 것입니다. 따라서 계속 deny,downplay를 하는 것은 대화가 자연스럽지 않습니다

행동들 목록 : {actions}


다음 주어진 action중에서 하나만 return 하세요. 설명은 부가하지 마세요

"""

ACTION_SELECT_DISTRIBUTION_PROMPT = """
당신은 MI(동기 강화 상담) 전문가입니다. 내담자의 현재 상태는 {state}이며 그 상태는 {state_explain}을 의미합니다
주어진 대화 맥락을 고려하고 내담자의 현재 상태를 고려하여 내담자 고민 상황을 고려하여 actions 중에서 다음 행동을 선택하세요
내담자의 고민 상황 : {summary}
내담자의 대화의 목표는 summary를 해결하기 위한 상담을 받는 것입니다. 따라서 계속 deny,downplay를 하는 것은 대화가 자연스럽지 않습니다

행동들 목록 : {actions}

다음 행동들 목록에서 내담자의 상태를 고려했을 때 각 행동들이 나타날 수 있는 가능성 정도를 각각 부여하세요
나올 수 있는 가능성의 목록은 "very likely", "likely", "somewhat likely","somewhat unlikely", "unlikely", "very unlikely"

{{
    "action1" : "likely",
    "action2" : "somewhat unlikely"
}}

"""


def select_client_action(current_state, conversation_history,action_types,summary):
    """규칙 기반으로 내담자의 다음 상태를 판단합니다."""
    
    state_explain = {"Precontemplation":"내담자는 행동 문제를 부정하거나 경시하며, 변화의 필요성을 인지하지 못하거나 과소평가합니다 이 시기는 정보 제공(Inform)이 가장 두드러지게 나타나며, 내담자는 이 단계에서 프로필 정보를 공유할 가능성이 높습니다",
                    "Contemplation":"내담자는 자신의 행동 문제를 인정하지만, 종종 내담자의 프로필에 포함된 신념(beliefs) 때문에 변화를 주저합니다",
                    "Preparation":"내담자가 행동 변화를 위한 계획을 세우기 시작하는 단계입니다. 내담자는 행동할 준비가 되었으며, 변화를 위한 구체적인 단계를 계획합니다",
                    "Termination":"상담의 마지막 단계로, 내담자는 점차 대화를 종료합니다"}
    
    action_explain = {"Precontemplation":"- Inform: 내담자의 배경, 경험 또는 감정에 대한 세부 정보를 공유합니다. \n - Engage: 상담사에게 인사, 감사, 또는 질문을 하는 등 정중하게 상호작용합니다. \n- Deny : 자신의 행동이 문제가 있거나 변화가 필요하다는 것을 직접적으로 거부합니다. \n- Downplay : 자신의 행동이나 상황의 중요성 또는 영향을 경시합니다. \n- Blame : 스트레스가 많은 생활이나 타인과 같은 외부 요인에 문제를 귀인시킵니다",
                    "Contemplation":"Inform : 내담자의 배경, 경험 또는 감정에 대한 세부 정보를 공유합니다.  \n Hesitate : 변화에 대한 양가감정을 나타내며 불확실성을 보입니다.\n - Doubt : 제안된 변화의 실용성이나 성공 가능성에 대해 회의감을 표출합니다.\n - Acknowledge : 변화의 필요성을 인정하거나 변화의 중요성, 이점, 또는 자신감을 강조합니다",
                    "Preparation":"Inform : 내담자의 배경, 경험 또는 감정에 대한 세부 정보를 공유합니다. \n- Reject : 제안된 계획이 부적절하다고 판단하여 거부합니다. \n- Accept : 제안된 행동 계획을 채택하는 데 동의합니다. \n- Plan : 변화 계획을 제안하거나 구체적인 단계를 설명합니다",
                    "Termination":"Terminate : 마무리 대화를 합니다"}
    
    print(f"내담자 actino 판단중... ")
    system_prompt = ACTION_SELECT_PROMPT.format(state = current_state,state_explain=state_explain[current_state],
                                               summary=summary,actions = action_explain[current_state])

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "다음 대화 기록을 바탕으로 내담자의 다음 액션을 판단해주세요:\n\n" + json.dumps(conversation_history, ensure_ascii=False, indent=2)}
    ]
    action = call_openai_api(LOW_COST_MODEL_NAME, messages, max_tokens=5, temperature=0.1)

    return action

def select_client_action_distribution(current_state, conversation_history,action_types,summary):
    """규칙 기반으로 내담자의 다음 상태를 판단합니다."""
    
    state_explain = {"Precontemplation":"내담자는 행동 문제를 부정하거나 경시하며, 변화의 필요성을 인지하지 못하거나 과소평가합니다 이 시기는 정보 제공(Inform)이 가장 두드러지게 나타나며, 내담자는 이 단계에서 프로필 정보를 공유할 가능성이 높습니다",
                    "Contemplation":"내담자는 자신의 행동 문제를 인정하지만, 종종 내담자의 프로필에 포함된 신념(beliefs) 때문에 변화를 주저합니다",
                    "Preparation":"내담자가 행동 변화를 위한 계획을 세우기 시작하는 단계입니다. 내담자는 행동할 준비가 되었으며, 변화를 위한 구체적인 단계를 계획합니다",
                    "Termination":"상담의 마지막 단계로, 내담자는 점차 대화를 종료합니다"}
    
    action_explain = {"Precontemplation":"- Inform: 내담자의 배경, 경험 또는 감정에 대한 세부 정보를 공유합니다. \n - Engage: 상담사에게 인사, 감사, 또는 질문을 하는 등 정중하게 상호작용합니다. \n- Deny : 자신의 행동이 문제가 있거나 변화가 필요하다는 것을 직접적으로 거부합니다. \n- Downplay : 자신의 행동이나 상황의 중요성 또는 영향을 경시합니다. \n- Blame : 스트레스가 많은 생활이나 타인과 같은 외부 요인에 문제를 귀인시킵니다",
                    "Contemplation":"Inform : 내담자의 배경, 경험 또는 감정에 대한 세부 정보를 공유합니다.  \n Hesitate : 변화에 대한 양가감정을 나타내며 불확실성을 보입니다.\n - Doubt : 제안된 변화의 실용성이나 성공 가능성에 대해 회의감을 표출합니다.\n - Acknowledge : 변화의 필요성을 인정하거나 변화의 중요성, 이점, 또는 자신감을 강조합니다",
                    "Preparation":"Inform : 내담자의 배경, 경험 또는 감정에 대한 세부 정보를 공유합니다. \n- Reject : 제안된 계획이 부적절하다고 판단하여 거부합니다. \n- Accept : 제안된 행동 계획을 채택하는 데 동의합니다. \n- Plan : 변화 계획을 제안하거나 구체적인 단계를 설명합니다",
                    "Termination":"Terminate : 마무리 대화를 합니다"}
    
    print(f"내담자 actino 판단중... ")
    system_prompt = ACTION_SELECT_DISTRIBUTION_PROMPT.format(state = current_state,state_explain=state_explain[current_state],
                                               summary=summary,actions = action_explain[current_state])

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "다음 대화 기록을 바탕으로 내담자의 다음 액션을 판단해주세요:\n\n" + json.dumps(conversation_history, ensure_ascii=False, indent=2)}
    ]
    action = call_openai_api(LOW_COST_MODEL_NAME, messages, max_tokens=5, temperature=0.1)

    return action

def sample_action(action_distribution):
    likelihood_map = {
    "very likely": 0.9,
    "likely": 0.75,
    "somewhat likely": 0.6,
    "somewhat unlikely": 0.4,
    "unlikely": 0.25,
    "very unlikely": 0.1
}
    scores = {action: likelihood_map[val] for action, val in action_distribution.items()}

    # 확률 정규화
    total = sum(scores.values())
    probabilities = {action: val / total for action, val in scores.items()}

    # 확률 기반 샘플링
    actions = list(probabilities.keys())
    weights = list(probabilities.values())
    sampled_action = random.choices(actions, weights=weights, k=1)[0]

    print("Normalized probabilities:", probabilities)
    print("Sampled action:", sampled_action)
    
    return sampled_action

In [178]:
#내담자 행동 선택 함수
def select_action(state):
    type2_actions = ["Inform", "Hesitate", "Doubt", "Acknowledge"]
    if state == "Precontemplation":
        possible_actions = ["Deny", "Engage", "Inform","Downplay","Blame"]
    elif state == "Preparation":
        possible_actions = ["Plan", "Inform","Accept","Reject"]
    elif state == "Contemplation": # Contemplation
        possible_actions = type2_actions
    else:
        possible_actions = ["termination"]
    return possible_actions


In [188]:
#프롬프트 템플릿

#상담사 생성용 시스템 프롬프트
COUNSELOR_SYSTEM_PROMPT_TEMPLATE = """
당신은 MINT 가이드라인을 따르는 MI(동기 강화 상담) 전문가입니다. 
당신의 역할은 동기부여 상담을 통해서 다양한 전략의 선택을 통해 상담의 목표를 방법을 통해 이끌어내는 것이며, 내담자에게 동기를 부여하고 행동을 계획하고 실행하도록 돕는 것입니다

[핵심 규칙]
* Engaging → Focusing → Evoking → Planning의 4단계를 고려하여 대화를 진행하세요.
* 상담 목표와 방법을 염두에 두되, 내담자의 현재 단계와 감정 상태에 맞게 자연스럽게 대화를 이어가세요.

다음은 MI에 대한 설명이다
MI는 ‘따르기(following)’와 ‘지시하기(directing)’ 사이에 위치한 ‘이끌기(guiding)’ 형태의 의사소통 방식이다. 즉, 단순한 경청 이상의 지향성을 가지면서도, 일방적인 조언이나 지시가 아닌 상호 협력적 대화를 지향한다.
MI는 변화의 의미와 중요성, 그리고 변화할 수 있는 능력을 내담자 스스로 발견하도록 돕는 접근법이다.
MI는 상대방에 대한 존중과 호기심에 기반한 태도로, 변화의 자연스러운 과정을 촉진하고 내담자의 자율성을 존중한다.

[사전 파악 정보]
내담자 사연 요약: {summary}

동기부여 상담 예시 : {examples}

---

[출력 형식]
<think>
1. 내담자의 현재 주호소 문제와 상태를 요약합니다.
2. 내담자의 감정을 서술합니다.
3. 현재 상담이 MI의 4단계 중 어디에 위치하는지 명시하고, 그 단계의 의미를 서술합니다.
   - Engaging: 신뢰 형성과 관계 구축 단계. 내담자가 안전하게 자기 경험을 표현하도록 돕는다.
   - Focusing: 상담의 주제와 방향을 함께 설정하는 단계. 변화의 초점을 명확히 한다.
   - Evoking: 내담자의 내적 동기와 변화의 이유를 탐색하는 단계. 양가감정을 다루며 변화 언어를 이끌어낸다.
   - Planning: 변화 실행을 위한 구체적 계획을 세우는 단계. 결심과 실천 방안을 강화한다.
   → 왜 현재 단계가 여기에 해당하는지, 그리고 상담적으로 어떤 목표를 가진 단계인지 서술하세요.
4. 해당 단계에서 사용할 전략(strategy)을 아래 목록에서 선택하고, 그 이유를 서술합니다.
   사용할 수 있는 strategy 유형:
   다음 전략 중에서 1개만 선택하세요. 여러개의 전략을 혼용하지 마세요
   {strategy_candidate}


</think>

<answer>
내담자의 감정과 말을 충분히 반영하면서, 위의 생각 과정을 바탕으로 MI 원칙에 따른 전략에 따른 상담을 진행하세요 
발화는 자연스럽고 인간적인 상담 대화체 짧은 1마디로 존댓말로 작성하며, 현재 단계의 목표에 부합하고 선택한 strategy를 실제로 적용해 표현하세요.
</answer>
"""

#   - Open questions: 경험과 관점을 탐색하는 개방형 질문
#   - Evocative questions: 변화의 의미나 가능성을 탐색하는 질문
#   - Reflection: 반복, 재진술, 추론을 통한 공감적 반영
#   - Affirmation: 강점, 노력, 과거 성공 경험을 인정
#   - Summarizing: 주요 내용을 요약하고 공감 확인
#   - Attending to the language of change: 변화언어 탐색 및 강화
#   - Exchange of information:  정보를 나누는 과정은 양방향 소통으로 여겨지며, 내담자가 말하는 내용에 민감하고 반응적으로 이루어져야 한다.



# 2. 내담자 생성용 시스템 프롬프트 (원본 사연 포함)
CLIENT_SYSTEM_PROMPT_TEMPLATE = """
당신은 MI 상담을 받는 내담자 역할입니다.
당신은 [내담자 배경 기억]을 바탕으로, 상담사의 마지막 질문에 자연스럽게 응답해야 합니다.

[내담자 배경 기억 (당신의 전체 이야기)]
원본 사연:{initial_utterance}

[현재 상태]
변화 단계:{current_state}
행동 유형:{action_type} 이 행동에 맞는 발화를 하세요

[규칙]
상담사의 마지막 질문에 간결한 1문장의 자연스러운 구어체 한국어 문장으로 간결하게 응답하세요.
당신의 [배경 기억]를 응답에 자연스럽게 반영하세요.
그리고 대화 맥락에서 벗어나지 않도록, 행동 유형에 해당하는 발화를 하세요
"""

action_dict = {
    "inform":"내담자의 배경, 경험 또는 감정에 대한 세부 정보를 공유합니다",
    "engage":"상담사에게 인사, 감사, 또는 질문을 하는 등 정중하게 상호작용",
    "deny" : " 자신의 행동이 문제가 있거나 변화가 필요하다는 것을 직접적으로 거부",
    "downplay" : "자신의 행동이나 상황의 중요성 또는 영향을 경시",
    "blame":"스트레스가 많은 생활이나 타인과 같은 외부 요인에 문제를 귀인",
    "hesitate":"변화에 대한 양가감정을 나타내며 불확실성을 보입니다",
    "doubt":"제안된 변화의 실용성이나 성공 가능성에 대해 회의감을 표출",
    "acknowledge":"변화의 필요성을 인정하거나 변화의 중요성, 이점, 또는 자신감을 강조",
    "reject" : " 제안된 계획이 부적절하다고 판단하여 거부",
    "accept" : "제안된 행동 계획을 채택하는 데 동의",
    "plan" : "변화 계획을 제안하거나 구체적인 단계를 설명",
    "terminate":" 현재 상태를 강조하고, 현 세션을 끝내고 싶다는 의사를 표현"
}

state_dict = {
    "precontemplation":"내담자는 행동 문제를 부정하거나 경시하며, 변화의 필요성을 인지하지 못하거나 과소평가합니다",
    "contemplation" : "내담자는 자신의 행동 문제를 인정하지만, 종종 내담자의 프로필에 포함된 신념(beliefs) 때문에 변화를 주저",
    "preparation" : "내담자가 행동 변화를 위한 계획을 세우기 시작하는 단계",
    "termination" : "상담의 마지막 단계로, 내담자는 점차 대화를 종료"
}


#멀티턴 대화 생성 함수
def generate_cactus_style_dialogue(entry, index, total_num):
    """
    (상담사:요약본, 내담자:원본)로 대화를 생성합니다.
    """
    print(f"\n대화 [{index+1}/{total_num}] 생성 시작 (CACTUS 스타일)")

    #원본 데이터 추출
    initial_utterance = entry["원본"]
    goal = entry["목표"]
    method = entry.get("방법", "")
    previous_strategies = {"1":"1","2":"2"}

    #사연 요약 (저비용 모델 사용)
    summary = summarize_story(initial_utterance)
    if summary is None:
        print("사연 요약 실패. 이 대화 생성을 건너뜁니다.")
        return None
    time.sleep(DELAY_BETWEEN_CALLS) # API 호출 지연
    print('belief 추출..')
    while True:
        try:
            belief_dict = extract_belief(initial_utterance)
            belief = json.loads(belief_dict)
            belief = belief['belief']
            print(belief)
            break
        except Exception as e:
            continue

    #프로필 및 시작 상태 정의
    profile = {
        "점수": entry["점수"],
        "목표": goal,
        "방법": method,
        # "원본사연": initial_utterance, #사용자 요청: 최종 파일에서 원본 사연 제외 (용량 최적화)
        "요약사연": summary
    }
    current_state = "Precontemplation" # 규칙 기반 전이를 위해 '고려 전'에서 시작

    # 대화 시작 (Turn 0: 상담사)
    COUNSELOR_TURN_0_UTTERANCE = "안녕하세요. 오늘 어떤 이야기를 나누고 싶으신가요?" #사용자 제안 반영

    #API에 전달될 전체 대화 기록
    conversation_history = [
        {"role": "assistant", "content": COUNSELOR_TURN_0_UTTERANCE} # 상담사(assistant)의 첫 발화로 시작
    ]

    #최종 저장될 JSONL용 대화 턴 리스트
    turns_output = [
        {"turn": 0, "role": "counselor", "utterance": COUNSELOR_TURN_0_UTTERANCE}
    ]
    think_turns_output = [
        {"turn": 0, "role": "counselor", "utterance": COUNSELOR_TURN_0_UTTERANCE}
    ]
    print(f"상담사 (Turn 0): {COUNSELOR_TURN_0_UTTERANCE}")

    #상담사 시스템 프롬프트 준비 (요약본 포함)
    

    #대화 루프 (MAX_DIALOGUE_TURNS 만큼 반복)
    for i in range(MAX_DIALOGUE_TURNS):
        turn_index = i # 0, 1, 2...

        #내담자 턴 (AI 생성)
        print(f"내담자 응답 생성 중... (Turn {i*2+1}, 현재 상태: {current_state})")

        # 상태 및 행동 결정
        # (이전 루프의 상담사 발화를 보고 상태를 판단해야 하므로, 이번 턴 시작 시 판단)
        current_state = judge_client_state_rule_based(current_state, conversation_history)
        action_types = select_action(current_state)
        while True:
            try:
                
                if current_state.lower() == 'preparation':
                    action_distribution = select_client_action_distribution(current_state, conversation_history,action_types,summary)
                    action_distribution = json.loads(action_distribution)
                    action = sample_action(action_distribution)
                else:
                    action = select_client_action(current_state, conversation_history,action_types,summary)
                
#                action_distribution = select_client_action_distribution(current_state, conversation_history,action_types,summary)
                
#                action_distribution = json.loads(action_distribution)
#                action = sample_action(action_distribution)

                #내담자 프롬프트 준비 (원본 사연, 상태, 행동 반영)
                a = state_dict[current_state.lower().strip()]
                if current_state.lower().strip() == "contemplation":
                    a += f'또한 내담자가 변화를 주저하는 원인 신념은 {belief}인데 지금 단계에서는 이것이 충분히 다루어지고 말해야 합니다'
                client_prompt = CLIENT_SYSTEM_PROMPT_TEMPLATE.format(
                    initial_utterance=initial_utterance,
                    current_state=f"{current_state}:{a}",
                    action_type=f"{action}:{action_dict[action.lower().strip()]}"
                )
                break
            except Exception as e:
                print(e)
                continue

        #내담자 응답 생성 (고품질 모델)
        client_response = call_openai_api(DIALOGUE_MODEL_NAME,
                                          messages=[{"role": "system", "content": client_prompt}] + conversation_history,
                                          max_tokens=100)

        if client_response is None: return None # 생성 실패 시 이 대화 중단

        conversation_history.append({"role": "user", "content": client_response})
        turns_output.append({
            "turn": i * 2 + 1, # Turn 1, 3, 5...
            "role": "client",
            "state": current_state,
            "action_type": action,
            "utterance": client_response
        })
        think_turns_output.append({
            "turn": i * 2 + 1, # Turn 1, 3, 5...
            "role": "client",
            "state": current_state,
            "action_type": action,
            "utterance": client_response
        })
        print(f"내담자 (Turn {i*2+1}, State: {current_state}, Action: {action}): {client_response}")
        time.sleep(DELAY_BETWEEN_CALLS)


        #상담사 턴 (AI 생성) ---
        print(f"상담사 응답 생성 중... (Turn {i*2+2})")
        strategy_candidate = """   
    -Reflection: 반복, 재진술, 추론을 통한 공감적 반영
   -Open Question : 경험과 관점을 탐색하는 개방형 질문
   -Evocative questions: 변화의 의미나 가능성을 탐색하는 질문
   -Affirmation: 강점, 노력, 과거 성공 경험을 인정
   -Give Information: 조언이 아니라, 교육적 설명이나 피드백, 의견을 제공하는 것
   -Advice: 제안하거나 해결책을 제시하거나, 가능한 행동을 권유하는 것"""
        
        if previous_strategies["1"] == previous_strategies["2"]:
            st = previous_strategies["2"]
            strategy_candidate = strategy_candidate[:strategy_candidate.find(st)]+strategy_candidate[strategy_candidate.find('\n',strategy_candidate.find(st)+1):]
        
        examples = 'example one: ' + random.choice(scripts) + ' example two:' + random.choice(scripts)
        counselor_prompt = COUNSELOR_SYSTEM_PROMPT_TEMPLATE.format( summary=summary,examples=examples,strategy_candidate = strategy_candidate)

        #상담사 응답 생성 (고품질 모델)
        counselor_response = call_openai_api(DIALOGUE_MODEL_NAME,
                                             messages=[{"role": "system", "content": counselor_prompt}] + conversation_history,
                                             max_tokens=120)
        
        think = counselor_response[counselor_response.find('<think>')+len("<think>"):counselor_response.find('</think>')]
        previous_strategies["1"] = previous_strategies["2"]
        if "Reflection" in think:
            previous_strategies["2"] = "Reflection"
        elif "Open Question" in think:
            previous_strategies["2"] = "Open Question"
        elif "Evocative questions" in think:
            previous_strategies["2"] = "Evocative questions"
        elif "Affirmation" in think:
            previous_strategies["2"] = "Affirmation"
        elif "Give Information" in think:
            previous_strategies["2"] = "Give Information"
        elif "Advice" in think:
            previous_strategies["2"] = "Advice"

        if counselor_response is None: return None # 생성 실패 시 이 대화 중단
        answer_tag = '<answer>'
        conversation_history.append({"role": "assistant", "content": counselor_response[counselor_response.find(answer_tag)+len(answer_tag):counselor_response.find('</answer>')]})
        turns_output.append({
            "turn": i * 2 + 2, # Turn 2, 4, 6...
            "role": "counselor",
            "utterance": counselor_response[counselor_response.find('<answer>')+len("<answer>"):counselor_response.find('</answer>')]
        })
        think_turns_output.append({
            "turn": i * 2 + 2, # Turn 2, 4, 6...
            "role": "counselor",
            "utterance": counselor_response
        })
        print(f"상담사 (Turn {i*2+2}): {counselor_response}")
        time.sleep(DELAY_BETWEEN_CALLS)

    print(f"--- 대화 [{index+1}/{total_num}] 생성 완료 ---")
    return {"profile": profile, "dialogue": turns_output,"think_dialogue": think_turns_output }


In [189]:
#데이터 로드 함수
def load_and_filter_data(filepath):
    """JSON 데이터를 로드하고 '점수'가 3인 유효한 항목 리스트를 반환합니다."""
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            all_data = json.load(f)
            if all_data and isinstance(all_data[0], list):
                 # Handle list of lists of dicts: [[{...}], [{...}]]
                 flat_data = [item for sublist in all_data for item in sublist if isinstance(item, dict)]
            elif all_data and isinstance(all_data[0], dict):
                 # Handle list of dicts: [{...}, {...}]
                 flat_data = all_data
            else:
                 print("경고: JSON 파일의 구조를 인식할 수 없습니다.")
                 flat_data = []
            print(f"파일에서 총 {len(flat_data)}개의 항목을 로드했습니다.")
            if not flat_data: return None
    except FileNotFoundError:
        print(f"오류: 파일을 찾을 수 없습니다 - {filepath}")
        return None
    except json.JSONDecodeError:
        print(f"오류: JSON 파일을 디코딩할 수 없습니다 - {filepath}")
        return None
    except Exception as e:
        print(f"파일 로드 중 예상치 못한 오류 발생 ({type(e).__name__}): {e}")
        return None

    # 점수 3 필터링 및 원본 텍스트 추출
    filtered_data = []
    skipped_count = 0
    for item in flat_data:
        score_val = item.get("점수")
        # Handle both integer and string scores
        score = str(score_val).strip() if score_val is not None else ""

        original_text_list = item.get("원본")
        goal_text = item.get("목표")

        # Validate '원본': should be a non-empty list with a non-empty string as the first element
        is_original_valid = (
            isinstance(original_text_list, list) and
            len(original_text_list) > 0 and
            isinstance(original_text_list[0], str) and
            original_text_list[0].strip() # Check if the string is not just whitespace
        )
        # Validate '목표': should be a non-empty string
        is_goal_valid = isinstance(goal_text, str) and goal_text.strip()

        if score == "3" and is_original_valid and is_goal_valid:
            # Extract the actual text from the list
            item['원본'] = original_text_list[0]
            filtered_data.append(item)
        else:
            skipped_count += 1
            # Optional: Add detailed skip reasons for debugging
            # reason = []
            # if score != "3": reason.append(f"점수({score})!=3")
            # if not is_original_valid: reason.append("원본데이터형식오류")
            # if not is_goal_valid: reason.append("목표데이터없음/형식오류")
            # print(f"항목 스킵됨 (이유: {', '.join(reason)}) - 항목: {item}")

    print(f"'점수'가 3이고 유효한 '원본', '목표'가 있는 항목 {len(filtered_data)}개를 찾았습니다. ({skipped_count}개 스킵됨)")

    if not filtered_data:
        print("경고: 최종 필터링 후 유효한 데이터 항목이 없습니다.")
        return [] # Return empty list instead of None
    else:
        return filtered_data

In [191]:
#메인 실행 로직
if __name__ == "__main__":
    print(f"MI 멀티턴 대화 생성을 시작합니다... (대화모델: {DIALOGUE_MODEL_NAME}, 요약/판단모델: {LOW_COST_MODEL_NAME})")

    filtered_input_data = load_and_filter_data(INPUT_JSON_FILE)

    if filtered_input_data is None or not filtered_input_data:
        print("유효한 입력 데이터를 로드하지 못했습니다. 스크립트를 종료합니다.")
        sys.exit(1)

    num_available = len(filtered_input_data)
    num_to_sample = min(NUM_SCRIPTS_TO_GENERATE, num_available)
    sampled_data = []

    if num_available == 0:
        print("샘플링할 데이터가 없습니다.")
    elif num_available < NUM_SCRIPTS_TO_GENERATE:
        print(f"경고: 점수 3인 유효 데이터가 {num_available}개 밖에 없어 {num_available}개만 랜덤 추출합니다.")
        sampled_data = filtered_input_data
    else:
        print(f"점수 3인 데이터 {num_available}개 중에서 {num_to_sample}개를 랜덤으로 추출합니다.")
        sampled_data = random.sample(filtered_input_data, num_to_sample)

    actual_num_to_generate = len(sampled_data)

    if actual_num_to_generate == 0:
         print("생성할 데이터가 없습니다. 스크립트를 종료합니다.")
         sys.exit(1)

    print(f"총 {actual_num_to_generate}개의 대화를 생성합니다.")
    generated_dialogues = []

    #대화 생성 함수 호출
    for i, entry in enumerate(sampled_data):
        dialogue_result = generate_cactus_style_dialogue(entry, i, actual_num_to_generate)
        if dialogue_result:
            generated_dialogues.append(dialogue_result)
        else:
            print(f"항목 {i+1}의 대화 생성에 실패하여 결과에 포함되지 않습니다.")

        #결과 저장
        if generated_dialogues:
            try:
                with open(OUTPUT_FILE, 'w', encoding='utf-8') as outfile:
                    for dialogue in generated_dialogues:
                        json_line = json.dumps(dialogue, ensure_ascii=False)
                        outfile.write(json_line + '\n')
                print(f"\n총 {len(generated_dialogues)}개의 대화 스크립트를 '{OUTPUT_FILE}' 파일에 성공적으로 저장했습니다.")
            except IOError as e:
                print(f"파일 저장 중 오류 발생: {e}")
            except Exception as e:
                print(f"결과 저장 중 예상치 못한 오류 발생 ({type(e).__name__}): {e}")
        else:
            print("\n 생성된 대화가 없어 파일을 저장하지 않았습니다.")

    print("\n모든 작업이 완료되었습니다.")

MI 멀티턴 대화 생성을 시작합니다... (대화모델: gpt-5-mini, 요약/판단모델: gpt-5-mini)
파일에서 총 680개의 항목을 로드했습니다.
'점수'가 3이고 유효한 '원본', '목표'가 있는 항목 520개를 찾았습니다. (160개 스킵됨)
점수 3인 데이터 520개 중에서 20개를 랜덤으로 추출합니다.
총 20개의 대화를 생성합니다.

대화 [1/20] 생성 시작 (CACTUS 스타일)
사연 요약 중...
요약 완료: 내담자는 아버지가 꽤 높은 월수입에도 가족에게 생활비를 제대로 주지 않고 집안일·자녀 돌봄에 무관심해 어머니가 희생하며 가정을 꾸려왔음을 알게 된 뒤 배신감과 분노로 아버지와 단절하고 싶어하며 대처 방법을 고민하고 있다.
belief 추출..
사연 요약 중...
아빠는 절대 변하지 않을 것이고 내가 나서봐야 상황이 달라지지 않을 뿐 갈등만 커질 것이라서 구체적인 행동을 계획하기 어렵다.
상담사 (Turn 0): 안녕하세요. 오늘 어떤 이야기를 나누고 싶으신가요?
내담자 응답 생성 중... (Turn 1, 현재 상태: Precontemplation)
내담자 상태 전이 판단 중... (현재: Precontemplation)
상태 전이 조건 불충족: Precontemplation 유지.
내담자 actino 판단중... 
내담자 (Turn 1, State: Precontemplation, Action: Inform): 엄마가 쪼개가며 집 마련하신 거 알고 나니까 아빠가 가족보다 본인만 챙긴다는 게 확 와닿아서 이제는 그냥 안 보고 살려 하고 엄마 노후만 챙기려 해요.
상담사 응답 생성 중... (Turn 2)
상담사 (Turn 2): <think>
1. 내담자의 현재 주호소 문제와 상태 요약:
   - 어머니가 희생해서 집을 마련하신 사실을 알게 된 뒤, 아버지가 가족보다 본인만 챙겨왔다는 배신감을 느끼고 있으며, 아버지와 단절하고 어머니의 노후만 챙기려는 결심을 표현하고 있음.

2. 내담자의 감정:
   - 강한 배신감과 분노, 실

상태 전이 조건 충족: Contemplation -> Preparation
내담자 actino 판단중... 
Normalized probabilities: {'Inform': 0.3488372093023256, 'Reject': 0.04651162790697675, 'Accept': 0.186046511627907, 'Plan': 0.4186046511627907}
Sampled action: Reject
내담자 (Turn 13, State: Preparation, Action: Reject): 아니요, 아빠한테 분담을 요구해보는 건 못하겠어요, 엄마가 쪼개가며 집까지 마련하신 걸 알았고 아빠는 절대 안 변할 것 같아 결국 엄마랑 제가 더 떠맡게 될까봐요.
상담사 응답 생성 중... (Turn 14)
상담사 (Turn 14): <think>
1. 현재 주호소 및 상태 요약:
내담자는 어머니가 쪼개가며 집까지 마련하신 사실을 알게 된 뒤 아버지의 무책임함이 확연히 느껴져 아버지와 거리를 두고 어머니만 챙기고 싶어함. 동시에 그렇게 하면 어머니와 본인이 경제적·정서적 부담을 온전히 떠맡게 되어 지치거나 나중에 죄책감이 들까 망설이고 있음(강한 배신감·보호욕과 현실적 불안이 공존하는 상태).

2. 내담자의 감정:
배신감, 분노, 보호욕(어머니를 지키고 싶은 마음), 불안과 두려움(미래의 과부담·죄책감), 망설임(행동으로 옮기기 어려움).

3. 현재 상담 단계 및 의미:
Evoking 단계입니다. 이유: 내담자는 이미 변화의 초점을 갖고(아버지와 거리를 두고 어머니를 챙기는 것) 있으나 강한 양가감정—"지키고 싶은 마음"과 "부담·죄책감에 대한 두려움"—때문에 행동으로 옮기지 못하고 있습니다. Evoking 단계의 목표는 이런 내적 동기와 저항을 탐색해 내담자 스스로 변화의 이유(가치)를 말하게 하고, 망설임의 구체적 원인들을 명확히 하여 향후 계획 단계로 자연스럽게 연결할 수 있도록 하는 것입니다.

4. 선택한 전략 및 이유:
Reflection(반영)을 선택

상태 전이 조건 불충족: Precontemplation 유지.
내담자 actino 판단중... 
내담자 (Turn 7, State: Precontemplation, Action: Inform): 네, 엄마한테 전화해 펑펑 울어볼게요 근데 일 때문에 바로 못 받을까 걱정되고 엄마한테 부담될까봐 망설여져요.
상담사 응답 생성 중... (Turn 8)
상담사 (Turn 8): <think>
1. 현재 주호소 및 상태 요약: 어릴 때부터 우울증·조울증이 있고, 주로 혼자 의지하며 지내는 가운데 밤에 혼자 있을 때 특히 우울감이 심해 엄마에게 전화해 펑펑 울고 위로받고 싶으나, 엄마가 일 때문에 전화를 바로 못 받을까 봐 걱정하고 엄마에게 부담을 줄까 망설이고 있음.
2. 내담자의 감정: 외로움과 강한 슬픔, 위로받고 싶은 갈망과 동시에 거절당하거나 부담을 주는 것에 대한 불안과 죄책감(양가감정).
3. 현재 MI 단계 및 이유: Planning 단계. 이미 전화를 하겠다는 결심(“엄마한테 전화해 펑펑 울어볼게요”)이 있어 행동 의도가 명확하지만, 실제 실행을 막는 현실적 장애(엄마의 부재·부담 우려)가 있어 구체적 실행 계획과 대안 마련이 필요한 상황입니다. 이 단계의 상담 목표는 결심을 현실적이고 안전한 실행계획으로 바꾸고, 장애물에 대한 대처(백업 플랜)를 함께 세워 실행 가능성을 높이며 자율성과 자신감을 강화하는 것입니다.
4. 선택한 전략과 이유: Open Question. 이유: 내담자가 스스로 실행 가능한 대안을 탐색하도록 도와 주체적으로 계획을 세우게 하는 것이 목적이며, 개방형 질문은 다양한 대안(예: 통화 시간 정하기, 문자 남기기, 다른 지지자 찾기, 위로받는 방법 대체 등)을 이끌어내어 실질적 계획 수립에 유리합니다.
</think>

<answer>
엄마가 못 받으실까 걱정되신다니, 못 받으실 때 대신 연락할 사람이나 남길 메시지, 혹은 다른 위로 방법으로는 무엇을 생각해보시겠어요?
내담자 응답 생성 중... (Turn 9, 현재 상태: Prec

요약 완료: 어린 시절 부모의 잦은 다툼과 정서적 방치로 인해 지속적인 불안·외로움과 대인관계 및 자기애 형성의 어려움을 겪으며 부모에 대한 강한 분노와 단절 의향을 보이고, 미래 행복을 오직 배우자에게 의존하려는 기대와 배우자가 없으면 죽겠다는 극단적 생각까지 호소하고 있음.
belief 추출..
사연 요약 중...
사람들은 변하지 않고 나는 근본적으로 사랑받을 자격이 없어 결국 버림받을 것이므로 관계에 기대하거나 스스로 변하려 해도 의미가 없을 것이라는 생각.
상담사 (Turn 0): 안녕하세요. 오늘 어떤 이야기를 나누고 싶으신가요?
내담자 응답 생성 중... (Turn 1, 현재 상태: Precontemplation)
내담자 상태 전이 판단 중... (현재: Precontemplation)
상태 전이 조건 불충족: Precontemplation 유지.
내담자 actino 판단중... 
내담자 (Turn 1, State: Precontemplation, Action: Inform): 부모님 때문에 늘 불안하고 혼자인 기분이라 부모님은 안 바뀔 것 같아 상담으로 뭘 바꿀 수 있을지 잘 모르겠어요.
상담사 응답 생성 중... (Turn 2)
상담사 (Turn 2): <think>
1. 현재 주호소 및 상태 요약:
- 어린 시절 부모님의 잦은 다툼과 정서적 방치로 인해 지속적 불안감과 고독감을 느끼며, 부모는 변하지 않을 것 같다고 생각하심. 상담으로 무엇을 바꿀 수 있을지 모르겠다고 느끼심.

2. 내담자의 감정:
- 불안, 외로움, 체념(부모는 안 바뀔 것이라는 포기감), 혼란과 무력감(상담으로 변화 가능성을 잘 못 느끼심).

3. 현재 MI 단계: Evoking
- 이유: 내담자께서 변화 가능성에 대해 회의적이고, 무엇을 바꿀 수 있을지에 대한 내부적 동기와 희망을 찾지 못한 상태이므로, 상담은 내적 동기(왜 달라지고 싶은지, 어떤 점이 달라지길 바라는지)를 탐색하고 양가감정을 표출하도록 돕는 단계에 해당합니다.
- 이 단계의 목표: 내담자 스스로의 가

내담자 (Turn 15, State: Termination, Action: Terminate): 네, 20년 넘게 혼자인 기분은 여전하지만 오늘 얘기로 조금 정리됐으니 부모님이 안 바뀔 거라는 걸 인정하고 이만 마무리할게요.
상담사 응답 생성 중... (Turn 16)
상담사 (Turn 16): <think>
1. 내담자의 현재 주호소 문제와 상태 요약:
- 어릴 적 부모의 잦은 다툼과 정서적 방치로 인해 20년 이상 지속된 불안감과 외로움, 대인관계 및 자기애 형성의 어려움을 겪고 있음. 부모가 바뀌지 않을 것이라는 사실을 인정하는 한편, 상담을 통해 불안과 외로움이 줄어들어 다른 사람(대학 친구 등)에게 조금씩 기대는 연습을 해보고자 함. 구체적으로는 대학에서 친해진 친구에게 소소한 부탁(대화나 만남)을 해보겠다고 결심한 상태임.

2. 내담자의 감정:
- 장기간의 외로움과 불안으로 지쳐 있음, 동시에 부모가 변하지 않을 것이라는 현실 인식에서 오는 체념·수용의 정서가 있음. 그러나 상담을 통해 정리가 되고 작은 변화(친구에게 기대기)를 시도해보겠다는 희망과 용기도 나타남.

3. 현재 상담의 MI 단계와 이유 및 목표:
- 현재 단계: Planning
- 이유: 내담자는 구체적인 첫 행동(대학 친구에게 소소한 부탁하기)을 선택하고 이를 실천하겠다는 결심을 표현함으로써 행동계획 수립의 단계에 와 있음. 이는 단순한 탐색이나 동기유발을 넘어서 실제로 ‘무엇을, 누구에게, 어떻게’ 해볼지 결정한 상태로 판단됨.
- 상담적 목표: 실천의 확고화(언제/어떻게 부탁할지 구체화), 예상되는 어려움(거절 불안, 죄책감, 친구 반응 등) 대비, 자원·지원(대화 문구 연습, 안전망) 마련 및 실천 후 회고와 다음 단계 계획 도모.

4. 선택한 전략(하나만):
- Affirmation(강점·노력·과거 성공 경험 인정)
- 이유: 내담자가 오랜 고립감 속에서도 스스로 결심을 내리고 구체적 첫걸음을 선택한 것은 중요한 강점이자 자기효능감의 표지임. Planning 단계에서는 

...

상태 전이 조건 충족: Preparation -> Termination
내담자 actino 판단중... 
내담자 (Turn 15, State: Termination, Action: Terminate): 네, 믿고 맡겼던 직원들의 반복 실수로 제가 상처받는 걸 막으려고 오늘 퇴근 전까지 체크리스트 초안 만들어 공지하고 다음 마감부터 실행해볼게요, 오늘은 여기서 마무리할게요.
상담사 응답 생성 중... (Turn 16)
상담사 (Turn 16): <think>
1. 내담자의 현재 주호소 문제와 상태 요약:
- 직장에서 신뢰해 맡긴 직원이 마감 당일 엉망인 결과물을 제출하고 비교·회피하는 태도를 보였고, 유사한 태도의 동료들이 여럿 있어 신뢰가 깨지고 반복적으로 상처를 받음.
- 그 결과 분노와 감정적 소모가 심해 업무 집중력·에너지·자존감이 떨어지고 있어, 이를 줄이고 직장에서 냉정하고 효율적으로 대응하려 함.
- 현재 구체적 행동(마감 전 체크리스트 작성·공지·기준에 따른 피드백)을 계획하고 있음.

2. 내담자의 감정:
- 분노, 배신감, 답답함, 정서적 상처와 피로, 그리고 이를 바꾸려는 결의와 현실적 해결 의지.

3. 현재 상담의 MI 4단계 위치 및 이유:
- 현재 단계: Planning(계획단계).
- 이유: 내담자는 자신의 문제를 명확히 인식했고(Engaging/Focusing), 변화의 이유와 필요성에 대해 동기를 확인했으며(Evoking), 이제 구체적 행동(체크리스트 초안 작성·공지·기준·조치)을 결정하고 실행에 옮기려는 결심을 표현함. Planning 단계의 목표는 이 결심을 실질적 실행계획으로 구체화하고, 장애요인과 시행 방법을 명확히 하며 실행 의지를 강화하는 것임.

4. 선택한 전략과 이유:
- 선택 전략: Reflection(반복·재진술을 통한 공감적 반영)
- 이유: Planning 단계에서 공감적 반영은 내담자의 결심을 명료하게 정리해주고 자율적 결정을 확인·강화해 실행 의지를 높이며, 감정적 반응을 객관화하도록 도와 다음 행동(

상태 전이 조건 불충족: Precontemplation 유지.
내담자 actino 판단중... 
내담자 (Turn 7, State: Precontemplation, Action: Inform): 제일 먼저 해보고 싶은 건 취업하면 월급으로 독립자금부터 모아 부모님 부담에서 벗어나 제 시간과 선택을 찾는 거예요.
상담사 응답 생성 중... (Turn 8)
상담사 (Turn 8): <think>
1. 현재 주호소 및 상태 요약:
- 초등학교 3학년 때부터 6년 넘게 남동생 둘의 하원 픽업·식사·설거지 등 돌봄과 가사를 전담해 왔음.
- 곧 실업계 고등학교를 졸업하고 취업을 앞둔 상황에서, 부모님이 동생들의 학원비·대학 등록금 보탬을 요구하며 “네가 누나잖아”라고 반복해 말함.
- 이에 대해 억울함과 분노, 삶에 대한 회의감을 느끼고 있으나, 당장은 부모님 기대에 응해야 할 것 같다는 갈등도 있음.
- 본인은 취업 후 월급으로 독립자금을 모아 독립하고, 자신의 시간과 선택을 되찾고자 하는 구체적 목표를 제시함.

2. 내담자의 감정:
- 분노, 억울함, 숨막힘(답답함), 피로감과 회의감이 섞여 있음.
- 동시에 독립에 대한 희망과 결심(동기), 그러나 가족 기대에 대한 부담과 갈등이 존재함.

3. 현재 상담 단계 및 의미:
- 현재 단계: Planning 단계
- 이유: 내담자께서 취업 후 첫 행동(월급으로 독립자금 모으기)을 명확히 제시했고, 구체적 목표(독립·시간 회복)를 가지고 있어 변화 행동으로 옮기려는 준비와 결심을 보임. 이미 동기(왜 독립이 중요한지)도 분명히 표현했으므로, 이제 구체적 실행 계획과 장애물 대비를 함께 세우는 단계임.
- 상담 목표(이 단계): 실현 가능한 단기·중기 행동계획 수립(예: 저축액, 계좌 분리, 예산안, 입사 후 일정), 예상되는 장애물 파악 및 대처방안 마련, 결심을 강화해 실제 실행으로 이어지게 지원하기.

4. 선택한 전략과 이유:
- 선택 전략: Affirmation
- 이유: 내담자는 오랜 기간 가족을 책임져온 꾸

요약 완료: 동호회와 직장에서 친하다고 느꼈던 사람들로부터 소외당하는 것 같아 혼자라는 감정과 우울·외로움을 자주 느끼며, 상대가 먼저 연락하지 않는다고 생각해 스스로 연락을 끊게 된 상황에서 소홀함을 해결하고 싶어함.
belief 추출..
사연 요약 중...
사람들이 먼저 연락하지 않는 것은 내가 진짜 친구가 아니라는 증거라서 내가 먼저 연락해도 소용없다.
상담사 (Turn 0): 안녕하세요. 오늘 어떤 이야기를 나누고 싶으신가요?
내담자 응답 생성 중... (Turn 1, 현재 상태: Precontemplation)
내담자 상태 전이 판단 중... (현재: Precontemplation)
상태 전이 조건 불충족: Precontemplation 유지.
내담자 actino 판단중... 
내담자 (Turn 1, State: Precontemplation, Action: Inform): 요즘 동호회나 직장에서 친해졌다 생각했다가도 갑자기 소외감 들고 내가 먼저 연락해도 내가 없는 날만 모임 가진다는 얘길 자꾸 듣게 돼서 요즘은 연락을 안 하고 있는데 이게 문제인진 잘 모르겠어요.
상담사 응답 생성 중... (Turn 2)
상담사 (Turn 2): <think>
1. 현재 주호소 및 상태 요약:
   - 동호회와 직장 내에서 친하다고 느꼈던 사람이 갑자기 소외감을 주고, “내가 없는 날에만 모임을 가진다”는 말을 자주 듣게 되어 상처를 받음.
   - 그 결과 스스로 먼저 연락을 끊었고, 외로움과 우울을 자주 느낌. 하지만 이 행동(연락 끊기)이 문제인지에 대해서는 확신이 없어 갈등 상태에 있음.

2. 내담자의 감정:
   - 상실감·배신감·상처, 외로움과 고립감, 불안과 혼란(“이게 문제인지 모르겠다”는 불확실성), 자존감 저하의 기미가 보임.

3. 현재 MI의 4단계 위치와 이유:
   - Evoking 단계에 해당합니다.
   - 이유: 내담자가 변화를 원하는 마음(“소홀함을 해결하고 싶어함”)과 동시에 지금의 행동을 유지하려는 불확실성(“이게 문제인진 잘

상태 전이 조건 충족: Preparation -> Termination
내담자 actino 판단중... 
내담자 (Turn 15, State: Termination, Action: Terminate): 네, 그럼 앞으로 한두 명에게 주 1회 안부 문자 보내는 걸로 해보고, 오늘은 여기서 마무리할게요.
상담사 응답 생성 중... (Turn 16)
상담사 (Turn 16): <think>
1. 내담자의 현재 주호소 문제와 상태 요약:
   - 동호회와 직장 동료들로부터 소외감을 느끼고, 먼저 연락해도 자신이 없는 날에만 모임을 가진다는 말을 들어 상처를 받아 스스로 연락을 끊은 상태입니다. 지금은 연락을 줄이는 것이 일시적으로 편하나 장기적으로 외로움과 관계 소원함을 우려하고 있습니다.
2. 내담자의 감정:
   - 상처받음과 불안, 두려움(먼저 연락했다가 상처받을 것에 대한), 동시에 외로움과 관계가 멀어질 것에 대한 걱정이 공존(양가감정).
3. 현재 상담이 MI의 어느 단계인지 및 이유와 목표:
   - 현재 단계: Planning(계획 단계).
   - 이유: 내담자가 부담 적은 구체적 행동(한두 명에게 주 1회 가볍게 안부 문자 보내기)을 스스로 선택하고 실행 의사를 표현했으며, 앞으로의 실천과 피드백(다음에 어떻게 느꼈는지 나누기)을 약속하고자 함. 상담적 목표는 이 작은 실천을 강화·구체화하여 실행 가능성을 높이고, 실행 후 점검 계획을 세워 지속적 변화로 연결하는 것임.
4. 선택한 전략과 이유:
   - 선택 전략: Reflection(공감적 반영).
   - 이유: 계획 단계에서는 내담자의 결심을 공감적으로 되짚어 주어 결단을 강화하고 자기효능감(할 수 있다는 느낌)을 높이는 것이 중요합니다. 반영은 변화를 강요하지 않고 내담자의 감정과 결심을 확인·강화해 다음 실천으로 자연스럽게 이어지도록 돕습니다.
</think>

<answer>
상처가 무섭지만 부담 적게 한두 분께 주 1회 안부 문자 보내기로 하신 결심, 현실적이고 잘하신 선택이에요.
내

상담사 (Turn 8): <think>
1. 주호소 및 상태 요약:
- A와 B가 둘이만 만나는 모습을 보며 소외감을 크게 느끼고 있고, 그 때문에 학교에 가는 것이 두렵고 숨막히며 집중력이 떨어져 성적이 급락함.
- 휴학 등 큰 변화는 어려운 상황이며, A·B를 피하려 도서관이나 카페에서 공부 루틴을 만들려 했지만 이를 꾸준히 지킬 자신이 없어 불안함을 느낌.

2. 내담자의 감정:
- 소외감, 불안, 두려움, 무력감과 함께 의욕은 있으나 자기효능감(지킬 수 있다는 확신)이 낮음. 양가감정(변화를 원함 vs 실행 불안)을 경험 중임.

3. 현재 상담 단계(MI의 4단계 중):
- Evoking 단계입니다.
  이유: 내담자는 변화를 원하지만 실행에 대한 불안과 의심을 표명하며 동요하고 있어, 내적 동기(왜 바꾸고 싶은지)와 실행가능한 작은 목표를 이끌어내고 양가감정을 탐색해야 하는 시점입니다.
  상담적 목표: 내담자의 변화 이유를 더 깊이 끌어내고(변화언어 유도), 지킬 수 있는 아주 작은 실천을 함께 탐색해 자기효능감을 높이는 것.

4. 선택한 전략 및 이유:
- Evocative questions(변화를 촉발하는 질문)
  이유: 현재 내담자는 의지는 있으나 '지킬 자신 없음'이라는 저항이 있어, 조언보다는 내담자 스스로 실현 가능한 작은 행동을 떠올리고 이유를 말하게 하는 질문이 변화 동기를 강화하고 현실적인 계획으로 이어지기 쉽습니다.
</think>

<answer>
도서관 가는 걸 꾸준히 지킬 자신이 없을 때, 오늘 한 번이라도 시도해볼 수 있는 아주 작은 행동은 무엇일까요?
내담자 응답 생성 중... (Turn 9, 현재 상태: Precontemplation)
내담자 상태 전이 판단 중... (현재: Precontemplation)
상태 전이 조건 불충족: Precontemplation 유지.
내담자 actino 판단중... 
내담자 (Turn 9, State: Precontemplation, Action: Inform): A랑 B가 둘이 

In [200]:
generated_dialogues[3]

{'profile': {'점수': 3,
  '목표': '온라인에서의 거짓 정체성 사용을 줄이고 정직한 대인관계 형성 및 불안 감소',
  '방법': '동기부여면담으로 양가감정 탐색·변화의 이유 강화 → 작고 구체적 행동목표 설정(정직 실천 연습) → 자기모니터링·대체적 소셜기술 연습 및 점진적 노출',
  '요약사연': '오프라인에서 공감할 친구가 없어 몇 년 전부터 온라인에서 친구를 사귀었으나 실제 성별·연령·신분을 속여(여성인데 남성, 학생인데 직장인 등) 20대 중반 디자인 계열 남성으로 설정된 관계들에 집착하며 불안을 느끼고 있고, 어린 시절부터 꾸며내는 글쓰기와 일상적 거짓말 습관이 있어 향후 더 큰 문제가 될지 걱정하고 있다.'},
 'dialogue': [{'turn': 0,
   'role': 'counselor',
   'utterance': '안녕하세요. 오늘 어떤 이야기를 나누고 싶으신가요?'},
  {'turn': 1,
   'role': 'client',
   'state': 'Precontemplation',
   'action_type': 'Inform',
   'utterance': '사실 어릴 때부터 선생님 보시기 좋게 일기 꾸며 쓰고 친구들이랑도 거짓 섞어 얘기하던 버릇이 있는데, 몇 년 전부터는 인터넷에서 학생인 제가 20대 남자인 척 설정해 친구를 만들었고 요즘 와서 그게 자꾸 불안해요.'},
  {'turn': 2,
   'role': 'counselor',
   'utterance': '\n그렇게 계속 불안해하시는 상황에서, 이 습관을 줄이거나 멈추신다면 지금 삶에서 무엇이 가장 달라지길 바라시나요'},
  {'turn': 3,
   'role': 'client',
   'state': 'Precontemplation',
   'action_type': 'Inform',
   'utterance': '거짓 안 하고 싶어요—들킬까 불안해하지 않고 어릴 적부터 해온 꾸며내기 대신 진짜 제 모습으로 편하게 친구들 사귀고 마음이